# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [309]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')  
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi.newsapi_client import NewsApiClient
from datetime import datetime, timedelta

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ironcommander\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [310]:
# Read your api key environment variable
load_dotenv()



True

In [311]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])


In [312]:
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(1), tz="America/New_York").isoformat()

# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()


In [313]:
def tokenizer(text):
    """Tokenizes text."""
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    # Create a tokenized list of the words
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    # Convert the words to lowercase & Remove the stop words
    tokens = [word.lower() for word in lem if word.lower() not in sw]
    return tokens


In [315]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
# also calculates the vader of each article 

def get_headlines(keyword):
    all_content=[]
    all_description=[]
    all_headlines=[]
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        content = []
        description = [] 
    
        title_sent=[]
        content_sent=[]
        descript_sent=[]
        all_title_sent=[]
        all_content_sent=[]
        all_descript_sent=[]
        title_tokens=[]
        content_tokens=[]
        descript_tokens=[]
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
            content.append(articles["articles"][i]["content"])
            description.append(articles["articles"][i]["description"])
        #vader sentiments
        #for headlines
            try:
                
                #title vader sentiments
                text = articles["articles"][i]["title"]
                sentiment = analyzer.polarity_scores(text)
                compound = sentiment["compound"]
                pos = sentiment["pos"]
                neu = sentiment["neu"]
                neg = sentiment["neg"]
        
                title_sent.append({
                    "text_title": text,
                    "date": date,
                    "compound_title": compound,
                    "positive_title": pos,
                    "negative_title": neg,
                    "neutral_title": neu,
                    "title_tokens":tokenizer(text)
                })
                all_title_sent.append(title_sent)


 #content vader sentiments
                text = articles["articles"][i]["content"]
                sentiment = analyzer.polarity_scores(text)
                compound = sentiment["compound"]
                pos = sentiment["pos"]
                neu = sentiment["neu"]
                neg = sentiment["neg"]
        
                content_sent.append({
                    "text_content": text,
                    
                    "compound_cont": compound,
                    "positive_cont": pos,
                    "negative_cont": neg,
                    "neutral_cont": neu,
                    "content_tokens":tokenizer(text)
                })
                all_content_sent.append(content.sent)

 #description vader sentiments
                text = articles["articles"][i]["description"]
                
                sentiment = analyzer.polarity_scores(text)
                compound = sentiment["compound"]
                pos = sentiment["pos"]
                neu = sentiment["neu"]
                neg = sentiment["neg"]
        
                descript_sent.append({
                    "text_descript": text,

                    "compound_descript": compound,
                    "positive_descript": pos,
                    "negative_descript": neg,
                    "neutral_descript": neu,
                    "descript_tokens":tokenizer(text)
                })
                all_descript_sent.append(descript_sent)


            
            except AttributeError:
                pass
    

        all_headlines.append(headlines)
        all_content.append(content)
        all_description.append(description)
        all_dates.append(date)
        date = date - timedelta(days=1)

         
       # Create DataFrame
    content_sentiment_df = pd.DataFrame(all_content_sent)
    title_sentiment_df = pd.DataFrame(all_title_sent)
    descript_sentiment_df = pd.DataFrame(all_descript_sent)
    final_df = title_sentiment_df.join(content_sentiment_df).join(descript_sentiment_df)
     # Reorder DataFrame columns
    cols = ["date", "text", "compound", "positive", "negative", "neutral"]
     #   df = df[cols]
    



    return all_headlines, all_dates, final_df
    

In [316]:
# Fetch the Bitcoin news articles
btc_headlines, btc_dates, btc_sentiments = get_headlines("bitcoin")
print(btc_articles)

Fetching news about 'bitcoin'
******************************
retrieving news from: 2022-02-12 00:00:00


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [ ]:
btc_sentiments.head()

,text_title,date,compound_title,positive_title,negative_title,neutral_title,text_content,compound_cont,positive_cont,negative_cont,neutral_cont,text_descript,compound_descript,positive_descript,negative_descript,neutral_descript,title_tokens
0,Jack Dorsey’s Block is working to make Bitcoin...,2022-01-14,-0.0258,0.204,0.212,0.584,Were officially building an open Bitcoin minin...,-0.0258,0.204,0.212,0.584,Block is working on building an “open Bitcoin ...,0.3182,0.133,0.074,0.793,b
1,Block is officially building an 'open Bitcoin ...,2022-01-14,-0.4404,0.000,0.195,0.805,Block founder Jack Dorsey has announced on Twi...,-0.4404,0.000,0.195,0.805,Block founder Jack Dorsey has announced on Twi...,-0.4404,0.000,0.068,0.932,b
2,Musk says Tesla to accept dogecoin for merchan...,2022-01-14,0.3818,0.271,0.000,0.729,Tesla Inc (TSLA.O) will accept the meme-based ...,0.3818,0.271,0.000,0.729,<table><tr><td></td></tr><tr><td> Representati...,0.3818,0.094,0.000,0.906,b
3,Jack Dorsey says Block is building a bitcoin m...,2022-01-14,-0.2500,0.121,0.222,0.657,Jack Dorsey onstage at a bitcoin convention in...,-0.2500,0.121,0.222,0.657,Block is officially building a bitcoin mining ...,-0.4404,0.000,0.172,0.828,b
4,Anthony Scaramucci says he's not buying the di...,2022-01-14,0.0000,0.000,0.000,1.000,Former Trump White House communications direct...,0.0000,0.000,0.000,1.000,The bitcoin price is likely to consolidate for...,0.0000,0.000,0.000,1.000,b


In [ ]:
# Fetch the Ethereum news articles
eth_headlines, eth_dates = get_headlines("ethereum")

Fetching news about 'ethereum'
******************************
retrieving news from: 2022-02-11 00:00:00
retrieving news from: 2022-02-10 00:00:00
retrieving news from: 2022-02-09 00:00:00


KeyboardInterrupt: 

In [ ]:
btc_headlines[0][19]

'With Valentine’s Day Approaching, the FBI Warns Issues a Scammer Alert'

In [ ]:
len(btc_headlines[20])

20

In [ ]:
eth_df.head()

In [ ]:
# Describe the Bitcoin Sentiment
btc_sentiments.describe()

,compound_title,positive_title,negative_title,neutral_title,compound_cont,positive_cont,negative_cont,neutral_cont,compound_descript,positive_descript,negative_descript,neutral_descript
count,20.000000,20.00000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000
mean,0.040480,0.11000,0.085250,0.804750,0.040480,0.11000,0.085250,0.804750,0.02287,0.074450,0.060200,0.865350
std,0.305733,0.11134,0.105671,0.131597,0.305733,0.11134,0.105671,0.131597,0.47172,0.082575,0.074619,0.102381
min,-0.440400,0.00000,0.000000,0.552000,-0.440400,0.00000,0.000000,0.552000,-0.83600,0.000000,0.000000,0.693000
25%,-0.250000,0.00000,0.000000,0.728500,-0.250000,0.00000,0.000000,0.728500,-0.44040,0.000000,0.000000,0.782000
50%,0.000000,0.12750,0.000000,0.800500,0.000000,0.12750,0.000000,0.800500,0.00000,0.058500,0.000000,0.873500
75%,0.323650,0.20575,0.188250,0.864750,0.323650,0.20575,0.188250,0.864750,0.34530,0.123250,0.134750,0.940750
max,0.542300,0.27300,0.293000,1.000000,0.542300,0.27300,0.293000,1.000000,0.92170,0.307000,0.210000,1.000000


In [ ]:
# Describe the Ethereum Sentiment
eth_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    # Create a tokenized list of the words
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    # Convert the words to lowercase & Remove the stop words
    tokens = [word.lower() for word in lem if word.lower() not in sw]
    return tokens



In [358]:
btc_tokens=[]
for i in range(0,len(btc_headlines)):
    for j in range(0,len(btc_headlines[i])):
        btc_tokens=btc_tokens+tokenizer(btc_headlines[i][j])
        
print(btc_tokens)

['took', 'days', 'netflix', 'order', 'docuseries', 'bonnie', 'clyde', 'crypto', 'world', 'bitcoin', 'paradise', 'briton', 'creates', 'crypto', 'utopia', 'south', 'pacific', 'cia', 'secretly', 'run', 'bulk', 'collection', 'program', 'youtube', 'exploring', 'web', 'onlyfans', 'debuts', 'nft', 'profile', 'pictures', 'weeks', 'business', 'crypto', 'roundup', 'obj', 'hope', 'come', 'rams', 'drake', 'uber', 'ceo', 'says', 'app', 'absolutely', 'accept', 'crypto', 'payment', 'future', 'galaxy', 'ha', 'builtin', 'cryptocurrency', 'wallet', 'bitcoin', 'v', 'sp', 'better', 'first', 'investment', 'crypto', 'suddenly', 'braced', 'huge', 'trillion', 'earthquake', 'extreme', 'volatility', 'hits', 'price', 'bitcoin', 'ethereum', 'bnb', 'solana', 'cardano', 'xrp', 'ui', 'business', 'alum', 'donates', 'bitcoin', 'fund', 'blockchain', 'education', 'champaignurbana', 'newsgazette', 'crypto', 'recovering', 'time', 'invest', 'bitcoin', 'hodler', 'accumulation', 'increasing', 'nationstate', 'bitcoin', 'strat

In [ ]:
# Create a new tokens column for Bitcoin


AttributeError: 'set' object has no attribute 'append'

In [ ]:
# Create a new tokens column for Ethereum
eth_tokens=[]
for i in range(0,len(eth_headlines)):
    for j in range(0,len(eth_headlines[i])):
        eth_tokens=eth_tokens+tokenizer(eth_headlines[i][j])
        
print(eth_tokens)

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_28000/39349498.py, line 2)

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [346]:
from collections import Counter
from nltk import ngrams


In [359]:
word_counts=Counter(btc_tokens)
print(dict(word_counts))

{'took': 2, 'days': 1, 'netflix': 5, 'order': 1, 'docuseries': 2, 'bonnie': 2, 'clyde': 2, 'crypto': 142, 'world': 6, 'bitcoin': 261, 'paradise': 3, 'briton': 1, 'creates': 2, 'utopia': 2, 'south': 1, 'pacific': 1, 'cia': 1, 'secretly': 1, 'run': 1, 'bulk': 1, 'collection': 2, 'program': 1, 'youtube': 2, 'exploring': 2, 'web': 6, 'onlyfans': 1, 'debuts': 1, 'nft': 12, 'profile': 5, 'pictures': 2, 'weeks': 3, 'business': 19, 'roundup': 4, 'obj': 1, 'hope': 2, 'come': 2, 'rams': 1, 'drake': 2, 'uber': 1, 'ceo': 5, 'says': 8, 'app': 2, 'absolutely': 2, 'accept': 5, 'payment': 9, 'future': 11, 'galaxy': 1, 'ha': 3, 'builtin': 1, 'cryptocurrency': 49, 'wallet': 7, 'v': 5, 'sp': 1, 'better': 8, 'first': 14, 'investment': 7, 'suddenly': 4, 'braced': 2, 'huge': 4, 'trillion': 4, 'earthquake': 1, 'extreme': 3, 'volatility': 4, 'hits': 2, 'price': 22, 'ethereum': 30, 'bnb': 5, 'solana': 9, 'cardano': 9, 'xrp': 4, 'ui': 1, 'alum': 1, 'donates': 1, 'fund': 6, 'blockchain': 17, 'education': 1, 'cha

In [361]:
def bigram_counter(corpus): 
    # Combine all articles in corpus into one large string
    big_string = ' '.join(corpus)
    processed = process_text(big_string)
    bigrams = ngrams(processed, n=2)
    top_10 = dict(Counter(bigrams).most_common(10))
    return pd.DataFrame(list(top_10.items()), columns=['bigram', 'count'])

# Generate the Bitcoin N-grams where N=2
bigram_counts = Counter(ngrams(btc_tokens, n=2))
print(dict(bigram_counts))

{('took', 'days'): 1, ('days', 'netflix'): 1, ('netflix', 'order'): 1, ('order', 'docuseries'): 1, ('docuseries', 'bonnie'): 1, ('bonnie', 'clyde'): 2, ('clyde', 'crypto'): 1, ('crypto', 'world'): 2, ('world', 'bitcoin'): 1, ('bitcoin', 'paradise'): 3, ('paradise', 'briton'): 1, ('briton', 'creates'): 1, ('creates', 'crypto'): 1, ('crypto', 'utopia'): 1, ('utopia', 'south'): 1, ('south', 'pacific'): 1, ('pacific', 'cia'): 1, ('cia', 'secretly'): 1, ('secretly', 'run'): 1, ('run', 'bulk'): 1, ('bulk', 'collection'): 1, ('collection', 'program'): 1, ('program', 'youtube'): 1, ('youtube', 'exploring'): 1, ('exploring', 'web'): 1, ('web', 'onlyfans'): 1, ('onlyfans', 'debuts'): 1, ('debuts', 'nft'): 1, ('nft', 'profile'): 2, ('profile', 'pictures'): 2, ('pictures', 'weeks'): 1, ('weeks', 'business'): 3, ('business', 'crypto'): 3, ('crypto', 'roundup'): 3, ('roundup', 'obj'): 1, ('obj', 'hope'): 1, ('hope', 'come'): 1, ('come', 'rams'): 1, ('rams', 'drake'): 1, ('drake', 'uber'): 1, ('uber'

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---